In [6]:
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)


def _save_json(p, obj):
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(p, "w") as f:
        json.dump(obj, f, indent=2)


def train_from_npy(
    train_npy_path,
    model_out_dir,
    train_labels_npy_path,
    random_seed=42,
    mem_limit_bytes=100_000_000_000,
    use_mmap=False,
    classifier='LogisticRegression'
):
    train_npy_path = Path(train_npy_path)
    if not train_npy_path.exists():
        raise FileNotFoundError(f"Train npy not found: {train_npy_path}")

    model_out_dir = Path(model_out_dir)
    model_out_dir.mkdir(parents=True, exist_ok=True)

    X_train = np.load(str(train_npy_path), mmap_mode="r" if use_mmap else None)
    if X_train.ndim != 2:
        raise RuntimeError(f"train array must be 2D. got shape={X_train.shape}")

    labels_path = Path(train_labels_npy_path)
    if not labels_path.exists():
        raise FileNotFoundError(f"Provided train_labels_npy_path does not exist: {labels_path}")
    y_train = np.load(str(labels_path), mmap_mode="r" if use_mmap else None)
    if y_train.shape[0] != X_train.shape[0]:
        raise RuntimeError(f"train labels length ({y_train.shape[0]}) != train rows ({X_train.shape[0]})")

    y_train = y_train.astype(np.int64)
    unique = np.unique(y_train)
    if unique.size != 2 or not np.array_equal(np.sort(unique), np.array([0, 1])):
        raise RuntimeError(f"train labels must contain exactly two classes 0 and 1. found: {unique}")
    y_train = y_train.astype(np.uint8)

    estimated_bytes = getattr(X_train, "nbytes", X_train.size * X_train.itemsize)
    if estimated_bytes > mem_limit_bytes:
        raise MemoryError(
            f"Estimated train bytes {estimated_bytes} > mem_limit_bytes {mem_limit_bytes}. "
            "Either increase mem_limit_bytes, reduce dataset size, or use use_mmap=True and implement chunked/out-of-core training."
        )

    scaler = MinMaxScaler()

    if classifier == 'LogisticRegression':
        print ('Using LogisticRegression as classifier')
        clf = LogisticRegression(random_state=random_seed, n_jobs=-1, verbose=1)
    elif classifier == 'RandomForest':
        print ('Using RandomForest as classifier')
        clf = RandomForestClassifier(random_state=random_seed, n_jobs=-1, verbose=1)
    else:
        print ('Using SVC as classifier')
        clf = SVC(random_state=random_seed, verbose=0)

    pipeline = Pipeline([("scaler", scaler), ("clf", clf)])

    print(f"Fitting pipeline")
    pipeline.fit(X_train, y_train)

    try:
        y_pred_train, y_score_train = _predict_and_score(pipeline, None, None, X_train)
        train_metrics = _compute_metrics(y_train, y_pred_train, y_score_train)

        print("TRAIN Eval results:")
        print(f"  acc={train_metrics['accuracy']:.4f}  prec={train_metrics['precision']:.4f}  "
              f"recall={train_metrics['recall']:.4f}  f1={train_metrics['f1']:.4f}")
        if train_metrics.get("roc_auc") is not None:
            print(f"  roc_auc={train_metrics['roc_auc']:.4f}")
        print("  confusion_matrix:", train_metrics["confusion_matrix"])
    except Exception as e:

        print("Warning: failed to compute train metrics:", e)
        train_metrics = None

    pipeline_path = model_out_dir / "pipeline.joblib"
    scaler_path = model_out_dir / "scaler.joblib"
    clf_path = model_out_dir / "clf.joblib"
    meta_out = model_out_dir / "train_meta.json"

    joblib.dump(pipeline, pipeline_path)
    joblib.dump(pipeline.named_steps["scaler"], scaler_path)
    joblib.dump(pipeline.named_steps["clf"], clf_path)

    meta_obj = {
        "train_npy": str(train_npy_path),
        "train_labels_npy": str(train_labels_npy_path),
        "train_rows": int(X_train.shape[0]),
        "C": None,
        "random_seed": int(random_seed),
        "use_mmap": bool(use_mmap),
        # store train metrics (or null) so you have a record
        "train_metrics": train_metrics,
    }
    _save_json(meta_out, meta_obj)

    print("Saved pipeline ->", pipeline_path)
    print("Saved scaler ->", scaler_path)
    print("Saved classifier ->", clf_path)
    return str(model_out_dir)

def _load_model(model_out_dir):
    """Load pipeline if present, else scaler+clf. Returns (pipeline, scaler, clf)."""
    model_out_dir = Path(model_out_dir)
    pipeline_path = model_out_dir / "pipeline.joblib"
    scaler_path = model_out_dir / "scaler.joblib"
    clf_path = model_out_dir / "clf.joblib"

    pipeline = None
    scaler = None
    clf = None

    if pipeline_path.exists():
        pipeline = joblib.load(pipeline_path)
    else:
        if scaler_path.exists():
            scaler = joblib.load(scaler_path)
        if clf_path.exists():
            clf = joblib.load(clf_path)
        if scaler is None or clf is None:
            raise FileNotFoundError("Could not find pipeline or scaler+clf in model_out_dir.")
    return pipeline, scaler, clf


def _compute_metrics(y_true, y_pred, y_score):
    metrics = {}
    metrics["accuracy"] = float(accuracy_score(y_true, y_pred))
    metrics["precision"] = float(precision_score(y_true, y_pred, zero_division=0))
    metrics["recall"] = float(recall_score(y_true, y_pred, zero_division=0))
    metrics["f1"] = float(f1_score(y_true, y_pred, zero_division=0))
    try:
        if y_score is not None and len(np.unique(y_true)) == 2:
            metrics["roc_auc"] = float(roc_auc_score(y_true, y_score))
        else:
            metrics["roc_auc"] = None
    except Exception:
        metrics["roc_auc"] = None
    metrics["confusion_matrix"] = confusion_matrix(y_true, y_pred).tolist()
    metrics["classification_report"] = classification_report(y_true, y_pred, zero_division=0, output_dict=True)
    return metrics


def _predict_and_score(pipeline, scaler, clf, X_test):
    y_score = None
    if pipeline is not None:
        y_pred = pipeline.predict(X_test)
        if hasattr(pipeline, "predict_proba"):
            try:
                y_score = pipeline.predict_proba(X_test)[:, 1]
            except Exception:
                y_score = None
        else:
            try:
                clf_ = pipeline.named_steps["clf"]
                scaler_ = pipeline.named_steps["scaler"]
                if hasattr(clf_, "decision_function"):
                    y_score = clf_.decision_function(scaler_.transform(X_test))
            except Exception:
                y_score = None
    else:
        X_t = scaler.transform(X_test)
        y_pred = clf.predict(X_t)
        if hasattr(clf, "predict_proba"):
            y_score = clf.predict_proba(X_t)[:, 1]
        elif hasattr(clf, "decision_function"):
            y_score = clf.decision_function(X_t)
    return y_pred, y_score


def eval_val_from_npy(
    val_npy_path,
    val_label_path,
    model_out_dir="./models",
    return_predictions=False,
    use_mmap=False,
    csv_path=None,  
):
    val_npy_path = Path(val_npy_path)
    val_label_path = Path(val_label_path)
    if not val_npy_path.exists():
        raise FileNotFoundError(f"val_npy not found: {val_npy_path}")

    X_test = np.load(str(val_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
    y_test = np.load(str(val_label_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
    
    if X_test.shape[0] != y_test.shape[0]:
        raise RuntimeError(f"val X rows ({X_test.shape[0]}) != val labels length ({y_test.shape[0]})")

    pipeline, scaler, clf = _load_model(model_out_dir)

    y_pred, y_score = _predict_and_score(pipeline, scaler, clf, X_test)

    metrics = _compute_metrics(y_test, y_pred, y_score)

    print("VAL Eval results:")
    print(f"  acc={metrics['accuracy']:.4f}  prec={metrics['precision']:.4f}  recall={metrics['recall']:.4f}  f1={metrics['f1']:.4f}")
    if metrics["roc_auc"] is not None:
        print(f"  roc_auc={metrics['roc_auc']:.4f}")
    print("  confusion_matrix:", metrics["confusion_matrix"])

    if csv_path is not None:
        csv_path = Path(csv_path)
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv(csv_path, index=False)

        if return_predictions:
            preds_path = csv_path.with_name(csv_path.stem + "_predictions.csv")
            preds_df = pd.DataFrame({
                "y_true": y_test,
                "y_pred": y_pred,
                "y_score": y_score
            })
            preds_df.to_csv(preds_path, index=False)
            print(f"Predictions saved to: {preds_path}")

        print(f"Metrics saved to: {csv_path}")

    return {
        "metrics": metrics,
        "y_true": y_test if return_predictions else None,
        "y_pred": y_pred if return_predictions else None,
        "y_score": y_score if return_predictions else None,
    }


def eval_test_from_npy(
    test_clean_npy_path,
    test_adv_npy_path=None,
    model_out_dir="./models",
    test_labels_npy_path=None,
    return_predictions=False,
    use_mmap=False,
    csv_path=None,  
):
    test_clean_npy_path = Path(test_clean_npy_path)
    if not test_clean_npy_path.exists():
        raise FileNotFoundError(f"test_clean npy not found: {test_clean_npy_path}")

    if test_adv_npy_path is None:
        if test_labels_npy_path is None:
            raise ValueError("Single-file mode requires test_labels_npy_path.")
        X_test = np.load(str(test_clean_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        y_test = np.load(str(test_labels_npy_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
        if X_test.shape[0] != y_test.shape[0]:
            raise RuntimeError(f"test X rows ({X_test.shape[0]}) != test labels length ({y_test.shape[0]})")
    else:
        test_adv_npy_path = Path(test_adv_npy_path)
        if not test_adv_npy_path.exists():
            raise FileNotFoundError(f"test_adv npy not found: {test_adv_npy_path}")
        X_clean = np.load(str(test_clean_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        X_adv = np.load(str(test_adv_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        X_test = np.vstack([X_clean, X_adv])
        if test_labels_npy_path is not None:
            y_test = np.load(str(test_labels_npy_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
            if y_test.shape[0] != X_test.shape[0]:
                raise RuntimeError(f"test labels length ({y_test.shape[0]}) != stacked test rows ({X_test.shape[0]})")
        else:
            y_test = np.concatenate([
                np.zeros(X_clean.shape[0], dtype=np.uint8),
                np.ones(X_adv.shape[0], dtype=np.uint8)
            ])

    pipeline, scaler, clf = _load_model(model_out_dir)
    y_pred, y_score = _predict_and_score(pipeline, scaler, clf, X_test)
    metrics = _compute_metrics(y_test, y_pred, y_score)

    print("TEST Eval results:")
    print(f"  acc={metrics['accuracy']:.4f}  prec={metrics['precision']:.4f}  recall={metrics['recall']:.4f}  f1={metrics['f1']:.4f}")
    if metrics["roc_auc"] is not None:
        print(f"  roc_auc={metrics['roc_auc']:.4f}")

    if csv_path is not None:
        csv_path = Path(csv_path)
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv(csv_path, index=False)

        if return_predictions:
            preds_path = csv_path.with_name(csv_path.stem + "_predictions.csv")
            preds_df = pd.DataFrame({
                "y_true": y_test,
                "y_pred": y_pred,
                "y_score": y_score
            })
            preds_df.to_csv(preds_path, index=False)
            print(f"Predictions saved to: {preds_path}")

        print(f"Metrics saved to: {csv_path}")

    return None



# Training

In [3]:
IMG_SIZE = 32
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'LogisticRegression'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    

Training: 32 - fgsm - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/fgsm-32-LogisticRegression/scaler.joblib
Saved classifier -> models/fgsm-32-LogisticRegression/clf.joblib
Training: 32 - bim - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/bim-32-LogisticRegression/scaler.joblib
Saved classifier -> models/bim-32-LogisticRegression/clf.joblib
Training: 32 - pgd - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


...

TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/pgd-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/pgd-32-LogisticRegression/scaler.joblib
Saved classifier -> models/pgd-32-LogisticRegression/clf.joblib
Training: 32 - df - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=0.9959  prec=1.0000  recall=0.9918  f1=0.9959
  roc_auc=0.9999
  confusion_matrix: [[486, 0], [4, 482]]
Saved pipeline -> models/df-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/df-32-LogisticRegression/scaler.joblib
Saved classifier -> models/df-32-LogisticRegression/clf.joblib
Training: 32 - cw - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=0.9938  prec=0.9959  recall=0.9918  f1=0.9938
  roc_auc=0.9999
  confusion_matrix: [[484, 2], [4, 482]]
Saved pipeline -> models/cw-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/cw-32-LogisticRegression/scaler.joblib
Saved classifier -> models/cw-32-LogisticRegression/clf.joblib


In [4]:
IMG_SIZE = 32
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'RandomForest'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    

Training: 32 - fgsm - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-32-RandomForest/pipeline.joblib
Saved scaler -> models/fgsm-32-RandomForest/scaler.joblib
Saved classifier -> models/fgsm-32-RandomForest/clf.joblib
Training: 32 - bim - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-32-RandomForest/pipeline.joblib
Saved scaler -> models/bim-32-RandomForest/scaler.joblib
Saved classifier -> models/bim-32-RandomForest/clf.joblib
Training: 32 - pgd - RandomForest 

Using RandomForest as classifier
Fitting pipeline


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.3s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/df-32-RandomForest/pipeline.joblib
Saved scaler -> models/df-32-RandomForest/scaler.joblib
Saved classifier -> models/df-32-RandomForest/clf.joblib
Training: 32 - cw - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.2s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/cw-32-RandomForest/pipeline.joblib
Saved scaler -> models/cw-32-RandomForest/scaler.joblib
Saved classifier -> models/cw-32-RandomForest/clf.joblib


[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [5]:
IMG_SIZE = 64
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'LogisticRegression'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    

Training: 64 - fgsm - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-64-LogisticRegression/pipeline.joblib
Saved scaler -> models/fgsm-64-LogisticRegression/scaler.joblib
Saved classifier -> models/fgsm-64-LogisticRegression/clf.joblib
Training: 64 - bim - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-64-LogisticRegression/pipeline.joblib
Saved scaler -> models/bim-64-LogisticRegression/scaler.joblib
Saved classifier -> models/bim-64-LogisticRegression/clf.joblib
Training: 64 - pgd - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


...

TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/pgd-64-LogisticRegression/pipeline.joblib
Saved scaler -> models/pgd-64-LogisticRegression/scaler.joblib
Saved classifier -> models/pgd-64-LogisticRegression/clf.joblib
Training: 64 - df - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=0.9979  prec=0.9979  recall=0.9979  f1=0.9979
  roc_auc=1.0000
  confusion_matrix: [[485, 1], [1, 485]]
Saved pipeline -> models/df-64-LogisticRegression/pipeline.joblib
Saved scaler -> models/df-64-LogisticRegression/scaler.joblib
Saved classifier -> models/df-64-LogisticRegression/clf.joblib
Training: 64 - cw - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/cw-64-LogisticRegression/pipeline.joblib
Saved scaler -> models/cw-64-LogisticRegression/scaler.joblib
Saved classifier -> models/cw-64-LogisticRegression/clf.joblib


In [6]:
IMG_SIZE = 64
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'RandomForest'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    

Training: 64 - fgsm - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.5s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-64-RandomForest/pipeline.joblib
Saved scaler -> models/fgsm-64-RandomForest/scaler.joblib
Saved classifier -> models/fgsm-64-RandomForest/clf.joblib
Training: 64 - bim - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.5s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-64-RandomForest/pipeline.joblib
Saved scaler -> models/bim-64-RandomForest/scaler.joblib
Saved classifier -> models/bim-64-RandomForest/clf.joblib
Training: 64 - pgd - RandomForest 

Using RandomForest as classifier
Fitting pipeline


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.5s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/df-64-RandomForest/pipeline.joblib
Saved scaler -> models/df-64-RandomForest/scaler.joblib
Saved classifier -> models/df-64-RandomForest/clf.joblib
Training: 64 - cw - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.6s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/cw-64-RandomForest/pipeline.joblib
Saved scaler -> models/cw-64-RandomForest/scaler.joblib
Saved classifier -> models/cw-64-RandomForest/clf.joblib


[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [7]:
IMG_SIZE = 128
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'RandomForest'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    print ('__________________________\n')

Training: 128 - fgsm - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    1.0s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-128-RandomForest/pipeline.joblib
Saved scaler -> models/fgsm-128-RandomForest/scaler.joblib
Saved classifier -> models/fgsm-128-RandomForest/clf.joblib
__________________________

Training: 128 - bim - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    0.8s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.5s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-128-RandomForest/pipeline.joblib
Saved scaler -> models/bim-128-RandomForest/scaler.joblib
Saved classifier -> models/bim-128-RandomForest/clf.joblib
__________________________

Training: 128 - pgd - RandomForest 

Using RandomForest as classifier
Fitting pipeline


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    1.3s remaining:   11.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.6s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/df-128-RandomForest/pipeline.joblib
Saved scaler -> models/df-128-RandomForest/scaler.joblib
Saved classifier -> models/df-128-RandomForest/clf.joblib
__________________________

Training: 128 - cw - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:    1.4s remaining:   12.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/cw-128-RandomForest/pipeline.joblib
Saved scaler -> models/cw-128-RandomForest/scaler.joblib
Saved classifier -> models/cw-128-RandomForest/clf.joblib
__________________________



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [8]:
IMG_SIZE = 128
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifier = 'LogisticRegression'
for ATTACK in ATTACKS:
    print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
    train_from_npy( train_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                    model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    train_labels_npy_path=f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                    random_seed=42,
                    mem_limit_bytes=100_000_000_000,
                    use_mmap=False,
                    classifier = classifier)
    print ('__________________________\n')

Training: 128 - fgsm - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/fgsm-128-LogisticRegression/pipeline.joblib
Saved scaler -> models/fgsm-128-LogisticRegression/scaler.joblib
Saved classifier -> models/fgsm-128-LogisticRegression/clf.joblib
__________________________

Training: 128 - bim - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/bim-128-LogisticRegression/pipeline.joblib
Saved scaler -> models/bim-128-LogisticRegression/scaler.joblib
Saved classifier -> models/bim-128-LogisticRegression/clf.joblib
__________________________

Training: 128 - pgd - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


...

TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/pgd-128-LogisticRegression/pipeline.joblib
Saved scaler -> models/pgd-128-LogisticRegression/scaler.joblib
Saved classifier -> models/pgd-128-LogisticRegression/clf.joblib
__________________________

Training: 128 - df - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=0.9990  prec=0.9979  recall=1.0000  f1=0.9990
  roc_auc=1.0000
  confusion_matrix: [[485, 1], [0, 486]]
Saved pipeline -> models/df-128-LogisticRegression/pipeline.joblib
Saved scaler -> models/df-128-LogisticRegression/scaler.joblib
Saved classifier -> models/df-128-LogisticRegression/clf.joblib
__________________________

Training: 128 - cw - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 96 concurrent workers.
/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TRAIN Eval results:
  acc=0.9990  prec=1.0000  recall=0.9979  f1=0.9990
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [1, 485]]
Saved pipeline -> models/cw-128-LogisticRegression/pipeline.joblib
Saved scaler -> models/cw-128-LogisticRegression/scaler.joblib
Saved classifier -> models/cw-128-LogisticRegression/clf.joblib
__________________________



# Evaluate Dataset (LR vs RF)

In [7]:
IMG_SIZES = [32, 64, 128]
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifiers = ['LogisticRegression', 'RandomForest']
for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        print (f'Attack Method: {ATTACK}\n')
        for classifier in classifiers:
            print (f'\n Classifier: {classifier}: \n')
            val_res = eval_val_from_npy(
                 val_npy_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/val_raw.npy",
                 val_label_path = f"/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/val_labels.npy",
                 model_out_dir = f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                 return_predictions=True,
                 use_mmap=False,
                 csv_path=f"./eval-VALSET/{ATTACK}-{ATTACK}-{IMG_SIZE}-{classifier}_test_result.csv"
            )
            print("val metrics:", val_res["metrics"])
        print ('_____________________\n')
    

Attack Method: fgsm


 Classifier: LogisticRegression: 

VAL Eval results:
  acc=0.8272  prec=0.8046  recall=0.8642  f1=0.8333
  roc_auc=0.8999
  confusion_matrix: [[64, 17], [11, 70]]
Predictions saved to: eval-VALSET/fgsm-fgsm-32-LogisticRegression_test_result_predictions.csv
Metrics saved to: eval-VALSET/fgsm-fgsm-32-LogisticRegression_test_result.csv
val metrics: {'accuracy': 0.8271604938271605, 'precision': 0.8045977011494253, 'recall': 0.8641975308641975, 'f1': 0.8333333333333334, 'roc_auc': 0.8998628257887518, 'confusion_matrix': [[64, 17], [11, 70]], 'classification_report': {'0': {'precision': 0.8533333333333334, 'recall': 0.7901234567901234, 'f1-score': 0.8205128205128205, 'support': 81.0}, '1': {'precision': 0.8045977011494253, 'recall': 0.8641975308641975, 'f1-score': 0.8333333333333334, 'support': 81.0}, 'accuracy': 0.8271604938271605, 'macro avg': {'precision': 0.8289655172413793, 'recall': 0.8271604938271604, 'f1-score': 0.8269230769230769, 'support': 162.0}, 'weighted a

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.8210  prec=0.7826  recall=0.8889  f1=0.8324
  roc_auc=0.9116
  confusion_matrix: [[61, 20], [9, 72]]
Predictions saved to: eval-VALSET/bim-bim-32-LogisticRegression_test_result_predictions.csv
Metrics saved to: eval-VALSET/bim-bim-32-LogisticRegression_test_result.csv
val metrics: {'accuracy': 0.8209876543209876, 'precision': 0.782608695652174, 'recall': 0.8888888888888888, 'f1': 0.8323699421965318, 'roc_auc': 0.9115988416399938, 'confusion_matrix': [[61, 20], [9, 72]], 'classification_report': {'0': {'precision': 0.8714285714285714, 'recall': 0.7530864197530864, 'f1-score': 0.8079470198675497, 'support': 81.0}, '1': {'precision': 0.782608695652174, 'recall': 0.8888888888888888, 'f1-score': 0.8323699421965318, 'support': 81.0}, 'accuracy': 0.8209876543209876, 'macro avg': {'precision': 0.8270186335403726, 'recall': 0.8209876543209876, 'f1-score': 0.8201584810320408, 'support': 162.0}, 'weighted avg': {'precision': 0.8270186335403727, 'recall': 0.82098765432098

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.8210  prec=0.7955  recall=0.8642  f1=0.8284
  roc_auc=0.8938
  confusion_matrix: [[63, 18], [11, 70]]
Predictions saved to: eval-VALSET/pgd-pgd-32-LogisticRegression_test_result_predictions.csv
Metrics saved to: eval-VALSET/pgd-pgd-32-LogisticRegression_test_result.csv
val metrics: {'accuracy': 0.8209876543209876, 'precision': 0.7954545454545454, 'recall': 0.8641975308641975, 'f1': 0.8284023668639053, 'roc_auc': 0.8937661941777167, 'confusion_matrix': [[63, 18], [11, 70]], 'classification_report': {'0': {'precision': 0.8513513513513513, 'recall': 0.7777777777777778, 'f1-score': 0.8129032258064516, 'support': 81.0}, '1': {'precision': 0.7954545454545454, 'recall': 0.8641975308641975, 'f1-score': 0.8284023668639053, 'support': 81.0}, 'accuracy': 0.8209876543209876, 'macro avg': {'precision': 0.8234029484029484, 'recall': 0.8209876543209876, 'f1-score': 0.8206527963351784, 'support': 162.0}, 'weighted avg': {'precision': 0.8234029484029483, 'recall': 0.8209876543

...

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.6296  prec=0.6364  recall=0.6049  f1=0.6203
  roc_auc=0.6586
  confusion_matrix: [[53, 28], [32, 49]]
Predictions saved to: eval-VALSET/df-df-128-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/df-df-128-RandomForest_test_result.csv
val metrics: {'accuracy': 0.6296296296296297, 'precision': 0.6363636363636364, 'recall': 0.6049382716049383, 'f1': 0.620253164556962, 'roc_auc': 0.6585886297820454, 'confusion_matrix': [[53, 28], [32, 49]], 'classification_report': {'0': {'precision': 0.6235294117647059, 'recall': 0.654320987654321, 'f1-score': 0.6385542168674698, 'support': 81.0}, '1': {'precision': 0.6363636363636364, 'recall': 0.6049382716049383, 'f1-score': 0.620253164556962, 'support': 81.0}, 'accuracy': 0.6296296296296297, 'macro avg': {'precision': 0.6299465240641711, 'recall': 0.6296296296296297, 'f1-score': 0.6294036907122159, 'support': 162.0}, 'weighted avg': {'precision': 0.6299465240641712, 'recall': 0.6296296296296297, 'f1-score

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.5926  prec=0.5882  recall=0.6173  f1=0.6024
  roc_auc=0.6251
  confusion_matrix: [[46, 35], [31, 50]]
Predictions saved to: eval-VALSET/cw-cw-128-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/cw-cw-128-RandomForest_test_result.csv
val metrics: {'accuracy': 0.5925925925925926, 'precision': 0.5882352941176471, 'recall': 0.6172839506172839, 'f1': 0.6024096385542169, 'roc_auc': 0.6251333638164914, 'confusion_matrix': [[46, 35], [31, 50]], 'classification_report': {'0': {'precision': 0.5974025974025974, 'recall': 0.5679012345679012, 'f1-score': 0.5822784810126582, 'support': 81.0}, '1': {'precision': 0.5882352941176471, 'recall': 0.6172839506172839, 'f1-score': 0.6024096385542169, 'support': 81.0}, 'accuracy': 0.5925925925925926, 'macro avg': {'precision': 0.5928189457601223, 'recall': 0.5925925925925926, 'f1-score': 0.5923440597834375, 'support': 162.0}, 'weighted avg': {'precision': 0.5928189457601223, 'recall': 0.5925925925925926, 'f1-sc

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [13]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/models")
output_zip = Path("/kaggle/working/models.zip")
manifest_path = Path("/kaggle/working/manifest-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/models.zip
  adding: cw-32-LogisticRegression/train_meta.json
  adding: cw-32-LogisticRegression/clf.joblib
  adding: cw-32-LogisticRegression/pipeline.joblib
  adding: cw-32-LogisticRegression/scaler.joblib
  adding: df-64-LogisticRegression/train_meta.json
  adding: df-64-LogisticRegression/clf.joblib
  adding: df-64-LogisticRegression/pipeline.joblib
  adding: df-64-LogisticRegression/scaler.joblib
  adding: bim-128-RandomForest/train_meta.json
  adding: bim-128-RandomForest/clf.joblib
  adding: bim-128-RandomForest/pipeline.joblib
  adding: bim-128-RandomForest/scaler.joblib
  adding: fgsm-32-LogisticRegression/train_meta.json
  adding: fgsm-32-LogisticRegression/clf.joblib
  adding: fgsm-32-LogisticRegression/pipeline.joblib
  adding: fgsm-32-LogisticRegression/scaler.joblib
  adding: bim-64-RandomForest/train_meta.json
  adding: bim-64-RandomForest/clf.joblib
  adding: bim-64-RandomForest/pipeline.joblib
  adding: bim-64-RandomForest/scaler.joblib
  

/tmp/ipykernel_10/3618908250.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")


Done.
Created: /kaggle/working/models.zip
Created: /kaggle/working/manifest-sha256.txt
Created: /kaggle/working/models.zip.sha256


In [8]:
classifier = ['LogisticRegression', 'RandomForest']
IMG_SIZES = [32, 64, 128]

ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
DATASETS = ['fgsm', 'bim', 'pgd', 'df', 'cw']

for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        for DATASET in DATASETS:
            for classifier in classifiers:
                print(f'Evaluate Against Test Set (Image Size: {IMG_SIZE}) \n Method: {ATTACK} \n Dataset: {DATASET} \n Classifier: {classifier}\n')
                eval_test_from_npy(
                    test_clean_npy_path=f'/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_clean_raw.npy',
                    test_adv_npy_path=f'/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_adv_raw.npy',
                    model_out_dir=f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    test_labels_npy_path=None,
                    return_predictions=False,
                    use_mmap=False,
                    csv_path=f"./eval/{ATTACK}-{DATASET}-{IMG_SIZE}-{classifier}_test_result.csv"
                )
                print ('_________________________\n')
        
            print ('===============================\n')




Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 
 Classifier: LogisticRegression

TEST Eval results:
  acc=0.7744  prec=0.7528  recall=0.8171  f1=0.7836
  roc_auc=0.8679
Metrics saved to: eval/fgsm-fgsm-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 
 Classifier: RandomForest

TEST Eval results:
  acc=0.7073  prec=0.6889  recall=0.7561  f1=0.7209
  roc_auc=0.7950
Metrics saved to: eval/fgsm-fgsm-32-RandomForest_test_result.csv
_________________________


Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 
 Classifier: LogisticRegression



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5427  prec=0.5686  recall=0.3537  f1=0.4361
  roc_auc=0.6792
Metrics saved to: eval/fgsm-bim-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 
 Classifier: RandomForest

TEST Eval results:
  acc=0.5427  prec=0.5556  recall=0.4268  f1=0.4828
  roc_auc=0.5927
Metrics saved to: eval/fgsm-bim-32-RandomForest_test_result.csv
_________________________


Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 
 Classifier: LogisticRegression



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5610  prec=0.5926  recall=0.3902  f1=0.4706
  roc_auc=0.6557
Metrics saved to: eval/fgsm-pgd-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 
 Classifier: RandomForest

TEST Eval results:
  acc=0.5854  prec=0.6000  recall=0.5122  f1=0.5526
  roc_auc=0.6191
Metrics saved to: eval/fgsm-pgd-32-RandomForest_test_result.csv
_________________________


Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: df 
 Classifier: LogisticRegression



...

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5671  prec=0.5647  recall=0.5854  f1=0.5749
  roc_auc=0.5848
Metrics saved to: eval/cw-df-128-RandomForest_test_result.csv
_________________________


Evaluate Against Test Set (Image Size: 128) 
 Method: cw 
 Dataset: cw 
 Classifier: LogisticRegression

TEST Eval results:
  acc=0.6220  prec=0.6087  recall=0.6829  f1=0.6437
  roc_auc=0.6773
Metrics saved to: eval/cw-cw-128-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 128) 
 Method: cw 
 Dataset: cw 
 Classifier: RandomForest



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5671  prec=0.5647  recall=0.5854  f1=0.5749
  roc_auc=0.6105
Metrics saved to: eval/cw-cw-128-RandomForest_test_result.csv
_________________________




[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [6]:
classifier = 'LogisticRegression'
IMG_SIZES = [32, 64, 128]

ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
DATASETS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifiers = ['LogisticRegression', 'RandomForest']

for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        for DATASET in DATASETS:
            for classifier in classifiers:
                print(f'Evaluate Against Test Set (Image Size: {IMG_SIZE}) \n Method: {ATTACK} \n Dataset: {DATASET} \n')
                eval_test_from_npy(
                    test_clean_npy_path=f'/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_clean_raw.npy',
                    test_adv_npy_path=f'/kaggle/input/layerpfs-dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_adv_raw.npy',
                    model_out_dir=f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                    test_labels_npy_path=None,
                    return_predictions=False,
                    use_mmap=False,
                    csv_path=f"./eval/{ATTACK}-{DATASET}-{IMG_SIZE}-{classifier}_test_result.csv"
                )
            print ('_________________________\n')
    
        print ('===============================\n')


Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 

TEST Eval results:
  acc=0.7744  prec=0.7528  recall=0.8171  f1=0.7836
  roc_auc=0.8679
Metrics saved to: eval/fgsm-fgsm-32-LogisticRegression_test_result.csv
Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.7073  prec=0.6889  recall=0.7561  f1=0.7209
  roc_auc=0.7950
Metrics saved to: eval/fgsm-fgsm-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 

TEST Eval results:
  acc=0.5427  prec=0.5686  recall=0.3537  f1=0.4361
  roc_auc=0.6792
Metrics saved to: eval/fgsm-bim-32-LogisticRegression_test_result.csv
Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 

TEST Eval results:
  acc=0.5427  prec=0.5556  recall=0.4268  f1=0.4828
  roc_auc=0.5927


[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


Metrics saved to: eval/fgsm-bim-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 

TEST Eval results:
  acc=0.5610  prec=0.5926  recall=0.3902  f1=0.4706
  roc_auc=0.6557
Metrics saved to: eval/fgsm-pgd-32-LogisticRegression_test_result.csv
Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 

TEST Eval results:
  acc=0.5854  prec=0.6000  recall=0.5122  f1=0.5526
  roc_auc=0.6191
Metrics saved to: eval/fgsm-pgd-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: df 



...

[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5671  prec=0.5647  recall=0.5854  f1=0.5749
  roc_auc=0.5848
Metrics saved to: eval/cw-df-128-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 128) 
 Method: cw 
 Dataset: cw 

TEST Eval results:
  acc=0.6220  prec=0.6087  recall=0.6829  f1=0.6437
  roc_auc=0.6773
Metrics saved to: eval/cw-cw-128-LogisticRegression_test_result.csv
Evaluate Against Test Set (Image Size: 128) 
 Method: cw 
 Dataset: cw 



[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5671  prec=0.5647  recall=0.5854  f1=0.5749
  roc_auc=0.6105
Metrics saved to: eval/cw-cw-128-RandomForest_test_result.csv
_________________________




[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 out of 100 | elapsed:    0.0s finished


In [9]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/eval")
output_zip = Path("/kaggle/working/evals.zip")
manifest_path = Path("/kaggle/working/manifest-eval-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/evals.zip
  adding: df-pgd-64-RandomForest_test_result.csv
  adding: pgd-bim-32-RandomForest_test_result.csv
  adding: df-fgsm-64-RandomForest_test_result.csv
  adding: cw-cw-64-LogisticRegression_test_result.csv
  adding: fgsm-pgd-128-LogisticRegression_test_result.csv
  adding: df-bim-64-RandomForest_test_result.csv
  adding: pgd-cw-128-LogisticRegression_test_result.csv
  adding: bim-pgd-32-LogisticRegression_test_result.csv
  adding: pgd-fgsm-32-LogisticRegression_test_result.csv
  adding: cw-fgsm-128-LogisticRegression_test_result.csv
  adding: pgd-bim-128-RandomForest_test_result.csv
  adding: bim-bim-128-LogisticRegression_test_result.csv
  adding: bim-df-64-LogisticRegression_test_result.csv
  adding: df-fgsm-128-LogisticRegression_test_result.csv
  adding: df-pgd-128-RandomForest_test_result.csv
  adding: df-pgd-32-LogisticRegression_test_result.csv
  adding: cw-bim-128-LogisticRegression_test_result.csv
  adding: fgsm-bim-128-LogisticRegression_t

/tmp/ipykernel_10/2260558218.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")


In [11]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/eval-VALSET")
output_zip = Path("/kaggle/working/evals-VALSET.zip")
manifest_path = Path("/kaggle/working/manifest-evals-VALSET-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/evals-VALSET.zip
  adding: cw-cw-64-LogisticRegression_test_result.csv
  adding: pgd-pgd-32-RandomForest_test_result_predictions.csv
  adding: fgsm-fgsm-32-LogisticRegression_test_result_predictions.csv
  adding: bim-bim-128-LogisticRegression_test_result.csv
  adding: pgd-pgd-64-LogisticRegression_test_result_predictions.csv
  adding: cw-cw-128-LogisticRegression_test_result_predictions.csv
  adding: fgsm-fgsm-64-RandomForest_test_result.csv
  adding: bim-bim-32-LogisticRegression_test_result.csv
  adding: pgd-pgd-64-LogisticRegression_test_result.csv
  adding: fgsm-fgsm-64-LogisticRegression_test_result.csv
  adding: cw-cw-32-LogisticRegression_test_result_predictions.csv
  adding: bim-bim-64-LogisticRegression_test_result_predictions.csv
  adding: pgd-pgd-128-LogisticRegression_test_result.csv
  adding: df-df-64-RandomForest_test_result.csv
  adding: fgsm-fgsm-128-RandomForest_test_result_predictions.csv
  adding: bim-bim-32-RandomForest_test_result.csv

/tmp/ipykernel_10/3829959068.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")


In [4]:
import shutil
from pathlib import Path
import os
from typing import Tuple

def clear_eval_dir(target: str = "/kaggle/working/eval") -> Tuple[int, int]:
    """
    Remove all items directly under `target` (keeps the target directory).
    No confirmation, no prompts.

    Returns:
        (removed_count, approx_removed_bytes)
    """
    p = Path(target)
    if not p.exists() or not p.is_dir():
        return 0, 0

    removed_count = 0
    removed_bytes = 0

    for child in p.iterdir():
        try:
            if child.is_dir() and not child.is_symlink():
                # estimate size (best-effort) then remove directory tree
                size = 0
                for root, _, files in os.walk(child, followlinks=False):
                    for f in files:
                        try:
                            size += (Path(root) / f).stat().st_size
                        except Exception:
                            pass
                shutil.rmtree(child)
                removed_bytes += size
            else:
                try:
                    removed_bytes += child.stat().st_size
                except Exception:
                    pass
                child.unlink()
            removed_count += 1
        except Exception:
            # ignore errors and continue
            continue

    return removed_count, removed_bytes

clear_eval_dir('/kaggle/working/eval-VALSET')


(300, 510185)